# Create an Ultra-long-life Small Modular Reactor Model with OpenMC
_The design recreated here is from [this paper](http://dx.doi.org/10.1016/j.anucene.2020.107390). This is an fast SMR with LBE coolant and U-Pu-Zr fuel._

This notebook creates a **voxel plot** and a **slice** of the created reactor. 
 
---
Note: The model may not completely represent the paper's one as the cladding fuel compostion and some other geometry considerations may not be represented here properly. So it is highly recommend to check everthing before using it.

### Add Necesssary Libraries
* OpenMC
* Matpltolib for inline plots

In [1]:
# add the necessary libraries
%matplotlib inline
import openmc as mc
#import numpy as np
import matplotlib.pyplot as plt

### Create Materials
1. upz:     U-Pu-Zr for fuel material which is a mixture of
    * Uranium
    * Plutonium
    * Zirconium
2. lbe:     Lead-Bismuth Eutectic for Coolant
3. steel:   15-15Ti Steel

Note: Note the reference paper for material composition.

In [2]:
uranium = mc.Material(1, "natural uranium")
uranium.add_nuclide('U238', 99.8, 'wo')
uranium.add_nuclide('U235', 0.2, 'wo')
uranium.add_nuclide('U234', 0.001, 'wo')
uranium.deplete = True

zirconium = mc.Material(2, "zirconium")
zirconium.add_element('Zr',1,'wo')

plutonium = mc.Material(3, "plutonium")
plutonium.add_nuclide('Pu238', 3.18, 'wo')
plutonium.add_nuclide('Pu239', 56.35, 'wo')
plutonium.add_nuclide('Pu240', 26.6, 'wo')
plutonium.add_nuclide('Pu241', 8.02, 'wo')
plutonium.add_nuclide('Pu242', 5.83, 'wo')
plutonium.deplete = True

upz = mc.Material.mix_materials([uranium, plutonium, zirconium], [0.8, 0.1, 0.1], 'wo')
upz.set_density('g/cc', 11.88)
upz.temperature = 700

In [3]:
lbe = mc.Material(5, "lead-bismuth-eutectic")

lbe.add_element('Pb', 44.5, 'wo')
lbe.add_element('Bi', 55.5, 'wo')

lbe.set_density('g/cc', 10.3)
lbe.temperature = 620.15

In [4]:
steel = mc.Material(6, "15-15Ti Steel")

steel.add_nuclide('C12', 0.090, 'wo')
steel.add_nuclide('Mn55', 1.502, 'wo')
steel.add_nuclide('Si28', 0.791, 'wo')
steel.add_nuclide('P31', 0.041, 'wo')

steel.add_element('Ti', 0.404, 'wo')
steel.add_element('Cr', 14.392, 'wo')
steel.add_element('Ni', 15.607, 'wo')
steel.add_element('B', 0.007, 'wo')
steel.add_element('Mo', 1.509, 'wo')

steel.set_density('g/cc', 7.92)
steel.temperature = 650

In [5]:
mats = mc.Materials([upz, lbe, steel])
mats.export_to_xml()

### Create Geometry
_This is a full core 3-D design with cylindrical fuel pin, hexagonal assembly lattice and the assemblies are also placed hexagonally in the core._  

---

**We are trying to reproduce this:**  
<img src="https://raw.githubusercontent.com/Fuad-HH/Conceptual-LBE-CooledSMR/main/gaoFullCore.png" width=600 height=600 />

#### First Create the Surfaces
Surfaces for:
1. ac_top & ac_bottom: To limit in the z-direction.
2. r_fuel: Cylinder for Outer Surface of Fuel
3. r_clad: Cylinder for Outer Surface of Cladding  

_There are 3 types of fuel pins with 3 different radii._

In [6]:
ac_top = mc.ZPlane(z0=70, boundary_type='vacuum')
ac_bottom = mc.ZPlane(z0=-70, boundary_type='vacuum')

f_radii = [0.74, 0.76, 0.78]
r_clad_radii = [0.77, 0.79, 0.81]

r_fuel = [mc.ZCylinder(r=ra) for ra in f_radii]
r_clad = [mc.ZCylinder(r=rc) for rc in r_clad_radii]

#### Create Cells and Universes for Fuel, Clad and Coolant
_Using list comprehension of Python to create multiple cells of varying sizes._

In [7]:
fuel = [mc.Cell(fill=upz, region=-cyl & +ac_bottom & -ac_top) for cyl in r_fuel]
clad = [mc.Cell(fill=steel, region=+f & -c & +ac_bottom & -ac_top) for f,c in zip(r_fuel,r_clad)]
coolant = [mc.Cell(fill=lbe, region=+c & -ac_top & +ac_bottom) for c in r_clad]
all_coolant_cell = mc.Cell(fill=lbe)

In [8]:
pin_universe = [mc.Universe(cells=[fuel[i],clad[i],coolant[i]], name='pin_for_zone '+str(i+1)) for i in [0,1,2]]
outer_universe = mc.Universe(cells=(all_coolant_cell,))
#outer_universe_for_assembly = mc.Universe(cells=(mc.Cell(fill=lbe, region=mc.model.hexagonal_prism(edge_length=16))))

### Create Fuel Assemblies with Fuel Pins of Varying Sizes
_Hexagonal lattice with 5 concentric hexagonal rings with 61 fuel pins._

In [9]:
lattice = [mc.HexLattice(name='assembly lattice:'+str(i+1)) for i in [0,1,2]]
    

In [10]:
for i in [0,1,2]:
    lattice[i].center = (0., 0.)
    lattice[i].pitch = (1.9, )
    lattice[i].outer = outer_universe
    lattice[i].orientation = 'x'
    rings = [[pin_universe[i]]*cir for cir in [24, 18, 12, 6, 1]]
    lattice[i].universes = rings

In [11]:
assem_region = mc.model.hexagonal_prism( edge_length=11.5, orientation='x', boundary_type='transmission')
assembly_cell = [mc.Cell(fill=lattice[i], region= assem_region & -ac_top & +ac_bottom) for i in [0,1,2]]
#outer_assembly_cell = mc.Cell(fill=lbe, region=-ac_top & +ac_bottom)

In [12]:
assembly_universe = [mc.Universe(cells=(assembly_cell[i],)) for i in [0, 1, 2]]

#### Create the Core Lattice
_The rings of assemblies are positioned manually as there are empty assembly positions as shown in the core design. There is also a ring of reflector._

In [13]:
core_lat = mc.HexLattice(name='core_lattice_hex')

core_lat.center = (0., 0.)
core_lat.pitch = (16,)
core_lat.outer = outer_universe


In [14]:
# the rings are created manually to make sure that the universes are in the correct order
reflective_ring = [outer_universe]*36
ring1 = [assembly_universe[2]]*30

ring2 = [assembly_universe[2]]*2 + [outer_universe] +\
    [assembly_universe[2]]*3 + [outer_universe] +\
    [assembly_universe[2]]*3 + [outer_universe] +\
    [assembly_universe[2]]*3 + [outer_universe] +\
    [assembly_universe[2]]*3 + [outer_universe] +\
    [assembly_universe[2]]*3 + [outer_universe] +\
    [assembly_universe[2]]

ring3 = [assembly_universe[1]]*18

ring4 = [outer_universe] + [assembly_universe[1]]*5 +\
        [outer_universe] + [assembly_universe[1]]*5

ring5 = [assembly_universe[0]]*6
ring6 = [assembly_universe[0]]
        

In [15]:
core_lat.universes = [reflective_ring, ring1, ring2, ring3, ring4, ring5, ring6]

#### Create Outside Boundary and Export the Geometry as XML

In [16]:
core_region = mc.model.hexagonal_prism(edge_length=110, orientation='y',boundary_type='vacuum')
core_cell = mc.Cell(fill=core_lat, region=core_region & -ac_top & +ac_bottom)
core_universe = mc.Universe(cells=(core_cell,))

In [17]:
geom = mc.Geometry()
geom.root_universe = core_universe
geom.export_to_xml()

### Create Voxel Plot
_You can open the produced `coreVoxel.vti` file with any vtk type file viewer such as `ParaView`._  

Note: This will create a .h5 file which is converted to .vti using openmc-voxel-to-vtk utility. If  you never used it before or the command is not working try `sudo apt-get install python-vtk` and try again.

In [18]:
# p_core for voxel plot
p_core = mc.Plot()
p_core.filename = 'coreVoxel'
p_core.color_by = 'material'
p_core.show_overlaps = True
p_core.type = 'voxel'

# here width and pixels are set to visualize approximately the center assembly.
# you can increase these values to visualize the whole core
p_core.width = (20, 20, 100)
p_core.pixels = (200, 200, 100)
p_core.depth = 5
p_core.highlight_domains(geom, [steel,lbe], seed=1, alpha=0.2, background='white')

# core_slice for slice plot of the core
core_slice = mc.Plot()
core_slice.filename = 'coreSlice'
core_slice.color_by = 'material'
core_slice.show_overlaps = True
core_slice.type = 'slice'
core_slice.width = (220, 220)
core_slice.pixels = (2000, 2000)
core_slice.basis = 'xy'

plots = mc.Plots([p_core, core_slice])
plots.export_to_xml()

mc.plot_geometry()

 Reading cross sections XML file...
 Reading materials XML file...
 Reading geometry XML file...
 Preparing distributed cell instances...
 Reading plot XML file...

 =======================>     PLOTTING SUMMARY     <========================

Plot ID: 1
Plot file: coreVoxel.h5
Universe depth: -1
Plot Type: Voxel
Origin: 0 0 0
Width:   20   20  100
Coloring: Materials
Voxels: 200 200 100

Plot ID: 2
Plot file: coreSlice.png
Universe depth: -1
Plot Type: Slice
Origin: 0 0 0
Width:  220  220
Coloring: Materials
Basis: XY
Pixels: 2000 2000

 Processing plot 1: coreVoxel.h5...
 Processing plot 2: coreSlice.png...


In [19]:
# convert .h5 to .vti file
!openmc-voxel-to-vtk coreVoxel.h5 -o coreVoxel.vti

Reading and translating data...
Writing VTK file coreVoxel.vti...


### Optional: Visulize Using Paraview and EOG

In [21]:
# visualize using paraview and eog (this is not necessary, but it given an idea of how to visualize the plots)
# otherwise run these commands on terminal

# change Coloring to id and Representation to Surface in paraview
!paraview coreVoxel.vti
!eog coreSlice.png

### Now Just Add Settings and Tallies to Simulate